# Compatible embeddings using Allombert's algorithm

In the following we represent $K\otimes C$ as $K[t]/\Phi(t)$ where $\Phi(t)$ is the polynomial defining 
the field $C = k(\zeta)$.

In [1]:
class ffembed:
    
    def __init__(self, k):
        self._field = k
        self._root = None
        self._hilbert_gen = None
    def __repr__(self):
        return self._field.__repr__()
    
    def has_root(self):
        return self._root != None
    
    def root(self):
        return self._root
    
    def hilbert_gen(self):
        return self._hilbert_gen
    
    def field(self):
        return self._field
    
    def assign_root(self, r, g):
        self._root = r
        self._hilbert_gen = g

class embedding:
    
    def __init__(self, c, f):
        self._constant = c
        self._map = f
        
    def __call__(self, x):
        return self._map(x)
    
    def constant(self):
        return self._constant
    
    def get_map(self):
        return self._map

In [2]:
"""
Compute the frobenius σ' of `elem` in K⊗C defined by σ'(x⊗y) = σ(x)⊗y with 
σ the frobenius of K/k with k the prime subfield.
"""
def froby(elem):
    l = elem.list()
    f = parent(l[0]).frobenius_endomorphism()
    res = parent(elem)()
    t = parent(elem).gen()
    for j in range(len(l)):
        res += f(l[j])*t^j
    return res  

"""
Compute the matrix representing the frobenius σ' in K⊗C, over the basis B⊗1
where B is a basis of K over k. The coefficients are in C=k(ζ).
"""
def frob_mat(K, C = None):
    
    if C == None:
        C = K.prime_subfield()
    
    frob = K.frobenius_endomorphism()
    n = K.degree()
    S = MatrixSpace(C, n)
    g = K.gen()
    
    cols = []
    
    for j in range(n):
        s = frob(g^j)
        l = s.polynomial().list()
        l = l + (n-len(l))*[C()]
        cols.append(l)
        
    return S(cols).transpose()

"""
Compute a solution of the equation ``σ'(x) = (1⊗ζ)x`` in K⊗C = T.
"""
def hilbert90(K, C, T):
    
    M = frob_mat(K, C)
    t = C.gen()
    eigen = M.right_eigenvectors()
    
    for e in eigen:
        if e[0] == t:
            v = e[1][0]
            break
    
    res = T()
    g = K.gen()
    for j in range(K.degree()):
        res += T(v[j])*g^j
        
    return res




In [10]:
"""
Compute the first coordinate of `elem` in `basis`, knowing that
`elem` is in fact in a subspace of degree `d2`.
"""
def change_basis(elem, basis, d2):
    
    k = elem.base_ring()
    d1 = elem.parent().degree()
    A = MatrixSpace(k, d1, d2)()
    
    for i in range(d2):
        L = (basis^i).list()
        for j in range(d1):
            A[j,i] = L[j]
            
    S2 = MatrixSpace(k, d1, 1)
    B = S2(elem.list())
    X = A.solve_right(B)
    
    return X[0,0]

def compute_root_no_cond(K1):
    
    k1 = K1.field()
    m = k1.degree()
    k = k1.prime_subfield()
    R = PolynomialRing(k, "x")
    f1 = factor(R(cyclotomic_polynomial(m)))[0][0]
    C1 = k.extension(f1, "t")
    T1 = k1.extension(f1, "t")
    a1 = hilbert90(k1, C1, T1)
    
    K1.assign_root(f1, a1)

    
def compute_root_compatible1(K2, K1):
    
    k1, k2 = K1.field(), K2.field()
    m, n = k1.degree(), k2.degree()
    k = k1.prime_subfield()
    R = PolynomialRing(k, "x")
    Rxy.<x,y> = PolynomialRing(k, "x, y")
    f1 = K1.root()
    f1xy = Rxy(f1)
    res = f1xy.resultant(y^(n/m)-x, x)
    P = gcd(R(res.polynomial(y).list()), R(cyclotomic_polynomial(n)))
    f2 = factor(P)[0][0]
    C2 = k.extension(f2, "t")
    T2 = k2.extension(f2, "t")
    a2 = hilbert90(k2, C2, T2)
    
    K2.assign_root(f2, a2)
        
def compute_root_compatible2(K1, K2):
    
    k1, k2 = K1.field(), K2.field()
    m, n = k1.degree(), k2.degree()
    k = k1.prime_subfield()
    R = PolynomialRing(k, "x")
    Rxy.<x,y> = PolynomialRing(k, "x, y")
    f2 = K2.root()
    f2xy = Rxy(f2)
    res = f2xy.resultant(x^(n/m)-y, x)
    P = gcd(R(res.polynomial(y).list()), R(cyclotomic_polynomial(m)))
    f1 = factor(P)[0][0]
    C1 = k.extension(f1, "t")
    T1 = k1.extension(f1, "t")
    a1 = hilbert90(k1, C1, T1)
    
    K1.assign_root(f1, a1)
        
def compute_root(K2, L = []):
    
    if type(L) != list:
        L = [L]
    
    if L == []:
        compute_root_no_cond(K2)
        
    elif len(L) == 1:
        if K2.field().degree() >= L[0].field().degree():
            compute_root_compatible1(K2, L[0])
        else:
            compute_root_compatible2(K2, L[0])
                
    else:
        
        k2 = K2.field()
        n = k2.degree()
        k = k2.prime_subfield()
        R = PolynomialRing(k, "x")
        Rxy.<x,y> = PolynomialRing(k, "x, y")
        P = R(cyclotomic_polynomial(n))
        
        for K1 in L:

            k1 = K1.field()
            m =  k1.degree()
            f1 = K1.root()
            f1xy = Rxy(f1)
            res = f1xy.resultant(y^(n/m)-x, x)
            P = gcd(R(res.polynomial(y).list()), P)
            
        f2 = factor(P)[0][0]    
        C2 = k.extension(f2, "t")
        T2 = k2.extension(f2, "t")
        a2 = hilbert90(k2, C2, T2)

        K2.assign_root(f2, a2)


In [4]:
"""
Compute two elements β and γ such that k1 = k(β) can be embedded
in k2 via the map β |-> γ. 
"""
def allombert_gens(k1, k2):
    
    # We first set some usefull variables
    k = k1.prime_subfield()
    R = PolynomialRing(k, "x")
    Rxy.<x,y> = PolynomialRing(k, "x, y")
    m, n = k1.degree(), k2.degree()
    
    # Then we create the spaces necessary to solve Hilbert 90
    # C1 = k(ζ1), T1 = k1 ⊗ C1
    # and we solve Hilbert 90 in T1
    f1 = factor(R(cyclotomic_polynomial(m)))[0][0]
    C1 = k.extension(f1, "t")
    T1 = k1.extension(f1, "t")
    a1 = hilbert90(k1, C1, T1)
    
    # After that we find a compatible polynomial f2 defining our root of unity ζ2
    f1xy = Rxy(f1)
    res = f1xy.resultant(y^(n/m)-x, x)
    P = gcd(R(res.polynomial(y).list()), R(cyclotomic_polynomial(n)))
    f2 = factor(P)[0][0]
    
    # We create the spaces relative to k2 and ζ2
    # and solve Hilbert 90 in T2 = k2 ⊗ k(ζ2)
    C2 = k.extension(f2, "t")
    T2 = k2.extension(f2, "t")
    a2 = hilbert90(k2, C2, T2)
    
    
    # We then coerce a1^m, which lives in T1, in C2 = k(ζ2) using
    # the embedding ζ1 |-> ζ2^(n/m) to see k(ζ1) in k(ζ2)
    b1 = C2([k(x) for x in (a1^m).list()]).polynomial().subs(C2.gen()^(n/m))
    
    # We coerce a2^n, which lives in T2, in C2 = k(ζ2)
    b2 = C2([k(x) for x in (a2^n).list()])
    
    # We compute an m-th root of b1/b2 = a1^m/a2^n
    c = (b1*b2^(-1)).nth_root(m)
    
    # And we finally return the first coefficient of a1 and c × a2 in
    # the base (1 ⊗ ζ2^(n/m)) 
    
    return a1.list()[0], change_basis(T2(c)*a2^(n/m), T2.gen()^(n/m), C1.degree())

"""
Compute the matrix whose columns are the powers of a form a^0 to a^(n-1).
"""
def basis_matrix(a, n = None):
    K = a.parent()
    m = K.degree()
    
    if n == None:
        n = m
    
    k = K.prime_subfield()
    S = MatrixSpace(k, m, n)()
    for j in range(n):
        L = (a^j).polynomial().list()
        i = 0
        for l in L:
            S[i, j] = l
            i += 1
    return S

def fflist(x):
    L = x.polynomial().list()
    l = len(L)
    k = x.parent()
    d = k.degree()
    if l < d:
        L += (d-l)*[k()]
        
    return L

"""
Compute the linear map φ: a|-> b sending a to b.
"""
def compute_map(a, b):
    
    A = basis_matrix(a)
    B = basis_matrix(b, a.parent().degree())
    C = B*A^(-1)
    
    K = b.parent()
    k = K.prime_subfield()
    S = MatrixSpace(k, a.parent().degree(), 1)

    return lambda x : K((C*S(fflist(x))).column(0))

def compute_embedding_no_cond(K1, K2):
    
    a1 = K1.hilbert_gen()
    a2 = K2.hilbert_gen()
    k1, k2 = K1.field(), K2.field()
    k = k1.prime_subfield()
    m, n = k1.degree(), k2.degree()
    C1 = k.extension(K1.root(), "t")
    C2 = k.extension(K2.root(), "t")
    T2 = a2.parent()
       
    b1 = C2([k(x) for x in (a1^m).list()]).polynomial().subs(C2.gen()^(n/m))
    b2 = C2([k(x) for x in (a2^n).list()])
    c = (b1*b2^(-1)).nth_root(m)
    
    a, b = a1.list()[0], change_basis(T2(c)*a2^(n/m), T2.gen()^(n/m), C1.degree())
    f = compute_map(a, b)
    return embedding(c, f)

def compute_embedding_compatible(K1, K2, f01, f02, K0):
    
    a1 = K1.hilbert_gen()
    a2 = K2.hilbert_gen()
    k1, k2 = K1.field(), K2.field()
    k = k1.prime_subfield()
    l, m, n = K0.field().degree(), k1.degree(), k2.degree()
    C1 = k.extension(K1.root(), "t")
    C2 = k.extension(K2.root(), "t")
    T2 = a2.parent()
       
    b1 = C2([k(x) for x in (a1^m).list()]).polynomial().subs(C2.gen()^(n/m))
    b2 = C2([k(x) for x in (a2^n).list()])
    e = (b1*b2^(-1)).nth_root(m)
    c, d = f01.constant().polynomial().subs(C2.gen()^(n/m)), f02.constant()
    
    zeta = (e^(m/l)*c*d^(-1)).nth_root(m/l)
    e = e*zeta^(-1)
    
    a, b = a1.list()[0], change_basis(T2(e)*a2^(n/m), T2.gen()^(n/m), C1.degree())
    f = compute_map(a, b)
    return embedding(e, f)

In [5]:
p = 5
k = GF(p)
k12 = GF(p^12)
k24 = GF(p^24)
k48 = GF(p^48)

In [4]:
a, b = allombert_gens(k12, k48)
a.minpoly() == b.minpoly()

True

In [6]:
K12 = ffembed(k12)
K24 = ffembed(k24)
K48 = ffembed(k48)

In [14]:
compute_root_no_cond(K12)
compute_root_compatible1(K48, K12)
f = compute_embedding_no_cond(K12, K48)
compute_root_compatible2(K24, K48)
g = compute_embedding_no_cond(K12, K24)
h = compute_embedding_compatible(K24, K48, g, f, K12)
hh = compute_embedding_no_cond(K24, K48)

In [11]:
compute_root(K12)
compute_root(K48, K12)
f = compute_embedding_no_cond(K12, K48)
compute_root(K24, K48)
g = compute_embedding_no_cond(K12, K24)
h = compute_embedding_compatible(K24, K48, g, f, K12)
hh = compute_embedding_no_cond(K24, K48)

In [12]:
z = k12.gen(); z

z12

In [13]:
f(z) == h(g(z))

True

In [14]:
f(z) == hh(g(z))

False

# Another case : losange

In [15]:
k7 = GF(p^7)
k21 = GF(p^21)
k14 = GF(p^14)
k42 = GF(p^42)

K7 = ffembed(k7)
K14 = ffembed(k14)
K21 = ffembed(k21)
K42 = ffembed(k42)

In [26]:
compute_root(K14)
compute_root(K21)
compute_root(K42, [K14, K21])

In [31]:
C.<t> = k.extension(K42.root(), "t")

In [32]:
(t^2).minpoly() == K21.root()

True

In [33]:
(t^3).minpoly() == K14.root()

True

# Tests